# Object Detection Demo
Welcome to the object detection inference walkthrough!  This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image. Make sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/object_detection/g3doc/installation.md) before you start.

# Imports

In [1]:
'''Append the path to TensorFlow object detection scripts'''
import sys, os

# Create a hi module in your home directory.
home_dir = os.path.expanduser("~/Documents/Metis/models/")

# # Add the home directory to sys.path
sys.path.append(home_dir)

In [1]:
import cv2
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
# import imutils
# import imageio

from collections import defaultdict
from io import StringIO
import matplotlib.pyplot as plt

'''
PIL is not needed as the video streams in OpenCV are already in numpy arrays 
(PIL is also a very big overhead specifically when using it for reading in 
the images aka video streams)
'''
# from PIL import Image

'\nPIL is not needed as the video streams in OpenCV are already in numpy arrays \n(PIL is also a very big overhead specifically when using it for reading in \nthe images aka video streams)\n'

In [2]:
# cap = cv2.VideoCapture(0) #("../../Main_Repo/Projects/HeliTrack/data/Neovision2-Training-Heli-001.mpg") #('vtest.avi')

# fgbg = cv2.createBackgroundSubtractorMOG2()

# while(1):
#     ret, frame = cap.read()

#     fgmask = fgbg.apply(frame)

#     cv2.imshow('frame',fgmask)
#     k = cv2.waitKey(30) & 0xff
#     if k == 27:
#         break

# cap.release()
# cv2.destroyAllWindows()

In [3]:
# vidcap = cv2.VideoCapture("data/Neovision2-Training-Heli-001.mpg") #('big_buck_bunny_720p_5mb.mp4')
# success,image = vidcap.read()
# count = 0
# success = True
# while success:
#   success,image = vidcap.read()
#   print('Read a new frame: ', success)
#   cv2.imwrite("/Users/aleksod/Downloads/frame%d.jpg" % count, image)     # save frame as JPEG file
#   count += 1

Read a new frame:  False


In [2]:
cap = cv2.VideoCapture("data/Neovision2-Training-Heli-001.mpg") #('vtest.avi')

while(cap.isOpened()):
    ret, frame = cap.read()
    
#     frame = imutils.resize(frame, width=600)
#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    cv2.imshow('frame',gray)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

## Env setup

In [2]:
# This is needed to display the images.
%matplotlib inline

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")

## Object detection imports
Here are the imports from the object detection module.

In [4]:
from utils import label_map_util

from utils import visualization_utils as vis_util

ImportError: cannot import name 'label_map_util'

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_CKPT` to point to a new .pb file.  

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [4]:
# What model to download.
MODEL_NAME = "faster_rcnn_resnet101_coco_11_06_2017" #'ssd_mobilenet_v1_coco_11_06_2017'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')

NUM_CLASSES = 90

In [5]:
print(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

http://download.tensorflow.org/models/object_detection/faster_rcnn_resnet101_coco_11_06_2017.tar.gz faster_rcnn_resnet101_coco_11_06_2017.tar.gz


## Download Model

In [6]:
'''Remove the model download part'''
# opener = urllib.request.URLopener()
# opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
# tar_file = tarfile.open(MODEL_FILE)
# for file in tar_file.getmembers():
#   file_name = os.path.basename(file.name)
#   if 'frozen_inference_graph.pb' in file_name:
#     tar_file.extract(file, os.getcwd())

## Load a (frozen) Tensorflow model into memory.

In [7]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [8]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

## Helper code

In [9]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Detection

In [10]:
# For the sake of simplicity we will use only 2 images:
# image1.jpg
# image2.jpg
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = 'test_images'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.jpg'.format(i)) for i in range(1, 3) ]

# Size, in inches, of the output images.
IMAGE_SIZE = (24, 16) #(12, 8)

In [11]:
'''
No “with” statement for the TensorFlow session as this is a huge overhead 
especially when every time the session needs to be started after each stream
'''
# with detection_graph.as_default():
#   with tf.Session(graph=detection_graph) as sess:
#     for image_path in TEST_IMAGE_PATHS:
#       image = Image.open(image_path)
#       # the array based representation of the image will be used later in order to prepare the
#       # result image with boxes and labels on it.
#       image_np = load_image_into_numpy_array(image)
#       # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
#       image_np_expanded = np.expand_dims(image_np, axis=0)
#       image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
#       # Each box represents a part of the image where a particular object was detected.
#       boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
#       # Each score represent how level of confidence for each of the objects.
#       # Score is shown on the result image, together with the class label.
#       scores = detection_graph.get_tensor_by_name('detection_scores:0')
#       classes = detection_graph.get_tensor_by_name('detection_classes:0')
#       num_detections = detection_graph.get_tensor_by_name('num_detections:0')
#       # Actual detection.
#       (boxes, scores, classes, num_detections) = sess.run(
#           [boxes, scores, classes, num_detections],
#           feed_dict={image_tensor: image_np_expanded})
#       # Visualization of the results of a detection.
#       vis_util.visualize_boxes_and_labels_on_image_array(
#           image_np,
#           np.squeeze(boxes),
#           np.squeeze(classes).astype(np.int32),
#           np.squeeze(scores),
#           category_index,
#           use_normalized_coordinates=True,
#           line_thickness=8)
#       plt.figure(figsize=IMAGE_SIZE)
#       plt.imshow(image_np)

'\nNo “with” statement for the TensorFlow session as this is a huge overhead \nespecially when every time the session needs to be started after each stream\n'